In [1]:
import torch
import os
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm_notebook
from tqdm import tqdm
from PIL import Image
from torchvision import datasets, models, transforms, models, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import lr_scheduler
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
#import torch_optimizer as optim
import random
#from ipywidgets import IntProgress
import time

In [2]:
# gpu or cpu
if torch.cuda.is_available():
  device = torch.device("cuda")
  # device name, number
  print(device,torch.cuda.get_device_name(0),torch.cuda.device_count())
else:
  device = torch.device("cpu")
  print(device)

cuda Tesla V100-SXM2-32GB 1


# Data Loader

In [3]:
df = pd.read_csv('./training_data_resize/training_data.csv')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 4 columns):
id              2028 non-null object
light           2028 non-null object
ground_truth    2028 non-null float64
grade           2028 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 63.5+ KB
None
                                        id light  ground_truth  grade
0  00130747_A_3457_20200715_100727_7.5.jpg     A           7.5      1
1  00130747_A_3458_20200715_100736_7.5.jpg     A           7.5      1
2  00130747_A_4810_20200812_112534_7.9.jpg     A           7.9      1
3  00130747_A_4811_20200812_112539_7.9.jpg     A           7.9      1
4  00241567_A_6659_20200916_153156_7.6.jpg     A           7.6      1


In [4]:
# class id
grade = df['grade'].to_numpy()
grade_set = set(grade)
grade_list = list(grade_set)
dic = {}
for i in range(len(grade_set)):
    dic[grade_list[i]] = i
print(dic)  # dic是一個class tpye to id

{1: 0, 2: 1, 3: 2}


In [5]:
# data, label
file_id = df["id"].to_numpy()
light = df["light"].to_numpy()
grade = df['grade'].to_numpy()

file = []
number = []
for i in range(len(light)):
    file.append(os.path.join("./training_data_resize", light[i], file_id[i]))
    number.append(dic[grade[i]])

'''
print(file)
print(number)
print(type(file))
print(type(number))
print(len(file))
print(len(number))
'''

# shuffle data
temp = list(zip(file, number))
random.shuffle(temp)
file, number = zip(*temp)

file = np.array(file)
number = np.array(number)

#train validation data 7:3
TVrate = 0.7
file_train = file[:int(len(file)*TVrate)]
file_vali = file[int(len(file)*TVrate):]
print(len(file_train))
#print(len(file_vali))

number_train = number[:int(len(file)*TVrate)]
number_vali = number[int(len(file)*TVrate):]
#print(number_train)

1419


In [6]:
preprocess = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(210),
    #transforms.Grayscale(num_output_channels=3),
    transforms.RandomAffine(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def default_loader(path):
    img_pil = Image.open(path)
    #img_pil = img_pil.resize((224, 224))
    img_tensor = preprocess(img_pil)
    return img_tensor

class train_set(Dataset):
    def __init__(self, loader=default_loader):
        self.images = file_train
        self.target = number_train
        self.loader = loader

    def __getitem__(self, index):
        fn = self.images[index]
        img = self.loader(fn)
        target = self.target[index]
        return img, target

    def __len__(self):
        return len(self.images)

class valid_set(Dataset):
    def __init__(self, loader=default_loader):
        self.images = file_vali
        self.target = number_vali
        self.loader = loader

    def __getitem__(self, index):
        fn = self.images[index]
        img = self.loader(fn)
        target = self.target[index]
        return img, target

    def __len__(self):
        return len(self.images)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [7]:
train_data = train_set()
trainloader = DataLoader(train_data, batch_size=64, shuffle=True,pin_memory=True)
valid_data = valid_set()
validloader = DataLoader(valid_data, batch_size=32, shuffle=True,pin_memory=True)
print(len(trainloader))
print(len(validloader))

23
20


# Model

In [8]:
'''
#from torchsummary import summary
model = models.vgg16(pretrained=True)
#model.classifier[6] = nn.Linear(4096,4096)
model.classifier[6] = nn.Linear(4096,3)

# resuming training
#model.load_state_dict(torch.load(img_path + '/vgg16_model/model_vgg'+ str(12)+'.pth'))
#summary(model.to('cuda:0'), (3, 224, 224))

#print(next(model.parameters()).is_cuda)
'''

"\n#from torchsummary import summary\nmodel = models.vgg16(pretrained=True)\n#model.classifier[6] = nn.Linear(4096,4096)\nmodel.classifier[6] = nn.Linear(4096,3)\n\n# resuming training\n#model.load_state_dict(torch.load(img_path + '/vgg16_model/model_vgg'+ str(12)+'.pth'))\n#summary(model.to('cuda:0'), (3, 224, 224))\n\n#print(next(model.parameters()).is_cuda)\n"

In [9]:

!pip install efficientnet-pytorch
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=3)

# resuming training
#model.load_state_dict(torch.load(img_path + '/modelb0_2/model'+ str(12)+'.pth'))
#summary(model.to('cuda:0'), (3, 224, 224))


Loaded pretrained weights for efficientnet-b2


In [10]:
'''
#from torchsummary import summary
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 3)
)
#summary(model.to('cuda:0'), (3, 224, 224))
'''

"\n#from torchsummary import summary\nmodel = models.resnet18(pretrained=True)\nmodel.fc = nn.Sequential(\n    nn.Dropout(0.5),\n    nn.Linear(512, 3)\n)\n#summary(model.to('cuda:0'), (3, 224, 224))\n"

In [11]:
'''
from torchsummary import summary
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096,3)

summary(model.to('cuda:0'), (3, 224, 224))
'''

"\nfrom torchsummary import summary\nmodel = models.alexnet(pretrained=True)\nmodel.classifier[6] = nn.Linear(4096,3)\n\nsummary(model.to('cuda:0'), (3, 224, 224))\n"

In [12]:
'''
# frozen model
from torchsummary import summary
model = models.vgg16(pretrained=True)

#for param in model.parameters():
#    param.requires_grad = False

#model.classifier[3] = nn.Linear(4096,4096)
model.classifier[6] = nn.Linear(4096,3)
#summary(model.to('cuda:0'), (3, 224, 224))
# resuming training
#model.load_state_dict(torch.load(img_path + 'vgg16_model/model_vgg5.pth'))
'''

"\n# frozen model\nfrom torchsummary import summary\nmodel = models.vgg16(pretrained=True)\n\n#for param in model.parameters():\n#    param.requires_grad = False\n\n#model.classifier[3] = nn.Linear(4096,4096)\nmodel.classifier[6] = nn.Linear(4096,3)\n#summary(model.to('cuda:0'), (3, 224, 224))\n# resuming training\n#model.load_state_dict(torch.load(img_path + 'vgg16_model/model_vgg5.pth'))\n"

In [13]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.4):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()

In [14]:
model = torch.nn.DataParallel(model, device_ids=None, output_device=None, dim=0).to(device)
#criterion = LabelSmoothingCrossEntropy()
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
#optimizer = optim.Ranger(params=model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
epoch_start = 1
epochs = 500
#unfrozen = 16

os.mkdir("./test")
for epoch in range(epoch_start, epochs+1):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    valid_loss_min = 0.0
    train_acc = 0
    val_acc = 0
    start = time.time()
    
    # write loss in txt file
    f = open('./test/model.txt', 'a')
    #f.write('running epoch: {}'.format(epoch))
    f.close()
    print('running epoch: {}'.format(epoch))

    '''
    ### unfrozen model  
    if epoch == unfrozen:
        for param in model.parameters():
            param.requires_grad = True
    if epoch == 30:
        optimizer = torch.optim.SGD(params=model.parameters(), lr=0.0001, momentum=0.9)
    '''

    ### train the model
    model.train()
    #for data, target in tqdm(trainloader):
    for data, target in trainloader:
        target = target.long()
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        #
        _, preds = torch.max(output, 1)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        train_acc += torch.sum(preds == target) 

    ### validate the model
    model.eval()
    #for data, target in tqdm(validloader):
    for data, target in validloader:
        target = target.long()
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        #
        _, preds = torch.max(output, 1)
        # update average validation loss
        valid_loss += loss.item()*data.size(0)
        val_acc += torch.sum(preds == target)
    # update lr
    #scheduler.step()
    
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)

    train_acc = int(train_acc)/int(len(file)*0.7)
    val_acc = int(val_acc)/int(len(file)*0.3)
    end = time.time()
    
    print('\tTraining Loss: {:.6f} \tTraining Acc: {:.4f} \tValidation Loss: {:.6f} \tValidation Acc: {:.4f} \tTime: {:.2f}'.format(
        train_loss, train_acc, valid_loss, val_acc, end-start))
    # write loss in txt file
    f = open('./test/model.txt', 'a')
    #f.write('\tTraining Loss: {:.6f} \tTraining Acc: {:.4f} \tValidation Loss: {:.6f} \tValidation Acc: {:.4f} \tTime: {:.2f}\n'.format(
    #    train_loss, train_acc, valid_loss, val_acc, end-start))
    f.close()

    # save model
    #torch.save(model.state_dict(),img_path + '/modelb_A_2/model'+ str(epoch) +'.pth')
    torch.save(model, './test/model'+ str(epoch) +'.pth')

FileExistsError: [Errno 17] File exists: './test'